<a href="https://colab.research.google.com/github/Nataliaricotorres/ean_PNL/blob/main/ejerciciobookingpln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # <span style="color:green">__Universidad EAN:</span> Facultad de Ingenieria - Maestría en Ciencias de Datos__
 ## **<span style="color:orange">Procesamiento de Lenguaje Natural</span>**

* __Geraldine Giseth Patiño Pardo__
* __Juan Sebastian Lozano Forero__
* __Natalia Rico Torres__

# __1. Importación y Carga de Librerias__

In [ ]:
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
import unicodedata
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy ## Procesamiento de lenguaje natural

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# __2. Carga de base de datos__

In [ ]:
## Se carga la base de datos
import pandas as pd

Booking_df=pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')
Booking_df

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25304\3885224651.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Booking_df=pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')


,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438208,438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,"9,0",NaN,NaN,No
438209,438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10,NaN,NaN,No
438210,438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,"1,0",No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10,NaN,NaN,No


# __3. Se generan Dataframes con reseñas de Colombia tanto positivas como negativas__

In [ ]:
# Filtrar y eliminar los registros donde 'reseña' sea "No" y 'País' sea "Colombia"
BookingCol = Booking_df[(Booking_df['reseña'] == 'Sí') & (Booking_df['País'] == 'Colombia')]

In [ ]:
BookingCol['Calificación']=BookingCol['Calificación'].str.replace(',','.').astype(float)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25304\3907479697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BookingCol['Calificación']=BookingCol['Calificación'].str.replace(',','.').astype(float)


In [ ]:
DF_inicial=BookingCol[BookingCol['Calificación']<=7]
DF_inicial=DF_inicial['Cosas Negativas']
DF_inicial=DF_inicial[DF_inicial.fillna('').apply(len)>60].sample(7500)
DF_inicial

87232     Dejé pago un servicio de transporte para el ae...
97797     Desde que llega uno lo hacen leer unas condici...
425113    el bar no estaba disponible, encontramos cabel...
297755    Se fue la luz y no servia el ascensor por casi...
9584      Podría mejorar las recomendaciones para los hu...
                                ...                        
360830    The buffet breakfast didn't really feel like a...
410879    Estaba el baño sucio, no tenía jabones de duch...
420494    Para tener en cuenta que es muy incómodo el no...
350703    Pague una avitacion por dos personas y solo no...
283905    La instalaciones del hotel son viejas y la hab...
Name: Cosas Negativas, Length: 7500, dtype: object

In [ ]:
DF_Final=pd.DataFrame()

DF_Final['Comentarios']=DF_inicial
DF_Final['Clas']='Negativos'
DF_Final

,Comentarios,Clas
87232,Dejé pago un servicio de transporte para el ae...,Negativos
97797,Desde que llega uno lo hacen leer unas condici...,Negativos
425113,"el bar no estaba disponible, encontramos cabel...",Negativos
297755,Se fue la luz y no servia el ascensor por casi...,Negativos
9584,Podría mejorar las recomendaciones para los hu...,Negativos
...,...,...
360830,The buffet breakfast didn't really feel like a...,Negativos
410879,"Estaba el baño sucio, no tenía jabones de duch...",Negativos
420494,Para tener en cuenta que es muy incómodo el no...,Negativos
350703,Pague una avitacion por dos personas y solo no...,Negativos


In [ ]:
DF_inicial2=BookingCol[BookingCol['Calificación']>=9]

DF_inicial2=DF_inicial2['Cosas Positivas']
DF_inicial2=DF_inicial2[DF_inicial2.fillna('').apply(len)>60].sample(7500)
DF_inicial2

398698    La atención!! Son muy agradables, serviciales ...
254855    La amabilidad del personal y las instalaciones...
359820    esta súper bien ubicado, es muy lindo y hay mu...
331826    La atención del personal, la privacidad y la h...
317335    El hotel está muy bien ubicado y cerca hay res...
                                ...                        
347392    Me encantó que me quedaba a solo unos pasos de...
427866    La cama fue muy comoda, por lo que pude descan...
112931    Hotel pequeño, ubicado cerca del centro comerc...
34714     Buenas habitaciones un poco pequeñas, limpio, ...
164633    Nos encanto la atencion, son personas muy amab...
Name: Cosas Positivas, Length: 7500, dtype: object

In [ ]:
DF_Final_2=pd.DataFrame()

DF_Final_2['Comentarios']=DF_inicial2
DF_Final_2['Clas']='Positivos'
DF_Final_2

,Comentarios,Clas
398698,"La atención!! Son muy agradables, serviciales ...",Positivos
254855,La amabilidad del personal y las instalaciones...,Positivos
359820,"esta súper bien ubicado, es muy lindo y hay mu...",Positivos
331826,"La atención del personal, la privacidad y la h...",Positivos
317335,El hotel está muy bien ubicado y cerca hay res...,Positivos
...,...,...
347392,Me encantó que me quedaba a solo unos pasos de...,Positivos
427866,"La cama fue muy comoda, por lo que pude descan...",Positivos
112931,"Hotel pequeño, ubicado cerca del centro comerc...",Positivos
34714,"Buenas habitaciones un poco pequeñas, limpio, ...",Positivos


In [ ]:
# Combinar los dataframes apilando filas
DF_BookingCol = pd.concat([DF_Final, DF_Final_2], ignore_index=True)

# Verificar el resultado
print(DF_BookingCol.shape)

(15000, 2)


# __4. A continuación se ejecutará cada los siguientes pasos dentro del procesamiento de texto:__
* __4.1. Normalización de texto__
* __4.2. Tokenización__
* __4.3. Eliminación de stopwords__
* __4.4. Stemming y lematización__
* __4.5. Bag of words__

## 4.1. Normalización de texto

In [ ]:
# Función para limpiar caracteres y eliminar tildes
def limpiar_caracteres_no_espanol_sin_tildes(texto):
    # Eliminar caracteres especiales y palabras en alfabetos no latinos
    texto = re.sub(r'[^\d\w\sñÑ]', '', texto)  # Solo deja palabras y espacios
    texto = re.sub(r'[^\u0000-\u007F\u00C0-\u017F\sñÑ]', '', texto)  # Filtrar caracteres no latinos

    # Normalizar y eliminar tildes
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'[\u0300-\u036f]', '', texto).strip()  # Eliminar marcas de acento (diacríticos)

    return texto

# Aplicar al DataFrame
DF_BookingCol['Texto_Limpio'] = DF_BookingCol['Comentarios'].apply(limpiar_caracteres_no_espanol_sin_tildes)

# Ver el resultado
print(DF_BookingCol[['Comentarios', 'Texto_Limpio']].head())


                                         Comentarios  \
0  Dejé pago un servicio de transporte para el ae...   
1  Desde que llega uno lo hacen leer unas condici...   
2  el bar no estaba disponible, encontramos cabel...   
3  Se fue la luz y no servia el ascensor por casi...   
4  Podría mejorar las recomendaciones para los hu...   

                                        Texto_Limpio  
0  Deje pago un servicio de transporte para el ae...  
1  Desde que llega uno lo hacen leer unas condici...  
2  el bar no estaba disponible encontramos cabell...  
3  Se fue la luz y no servia el ascensor por casi...  
4  Podria mejorar las recomendaciones para los hu...  


In [ ]:
# Función actualizada para eliminar números y normalizar texto
def normalizar_texto_completamente(texto):
    if isinstance(texto, str):  # Asegurarse de procesar solo texto
        # Eliminar números
        texto = re.sub(r'\d', '', texto)
        # Eliminar caracteres especiales y normalizar tildes
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        # Eliminar espacios adicionales
        texto = re.sub(r'\s+', ' ', texto).strip()
        return texto
    return texto  # Si no es texto, devolver sin cambios

# Aplicar la función de limpieza
DF_BookingCol['Texto_Limpio'] = DF_BookingCol['Texto_Limpio'].apply(normalizar_texto_completamente)

## 4.2. Tokenización

In [ ]:
# Asegúrate de descargar el recurso necesario
nltk.download('punkt')

# Aplicar la tokenización por palabras a cada fila de la columna 'Comentarios'
DF_BookingCol['Tokens'] = DF_BookingCol['Texto_Limpio'].apply(word_tokenize)

# Ver el resultado
print(DF_BookingCol[['Comentarios', 'Texto_Limpio', 'Tokens']])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                             Comentarios  \
0      Dejé pago un servicio de transporte para el ae...   
1      Desde que llega uno lo hacen leer unas condici...   
2      el bar no estaba disponible, encontramos cabel...   
3      Se fue la luz y no servia el ascensor por casi...   
4      Podría mejorar las recomendaciones para los hu...   
...                                                  ...   
14995  Me encantó que me quedaba a solo unos pasos de...   
14996  La cama fue muy comoda, por lo que pude descan...   
14997  Hotel pequeño, ubicado cerca del centro comerc...   
14998  Buenas habitaciones un poco pequeñas, limpio, ...   
14999  Nos encanto la atencion, son personas muy amab...   

                                            Texto_Limpio  \
0      Deje pago un servicio de transporte para el ae...   
1      Desde que llega uno lo hacen leer unas condici...   
2      el bar no estaba disponible encontramos cabell...   
3      Se fue la luz y no servia el asc

## 4.3. Eliminación de stopwords

In [ ]:
# Stopwords en español e inglés como conjuntos
stopwords_esp = set(stopwords.words('spanish'))  # Convertir a conjunto
stopwords_eng = set(stopwords.words('english'))  # Convertir a conjunto

# Combinar ambas listas de stopwords
stopwords_combined = stopwords_esp.union(stopwords_eng)  # Unir ambos conjuntos

# Verificar la combinación
print(f"Total stopwords combinadas: {len(stopwords_combined)}")

# Eliminar stopwords en ambos idiomas
DF_BookingCol['Sin_Stopwords_Combined'] = DF_BookingCol['Tokens'].apply(
    lambda tokens: [word for word in tokens if word.lower() not in stopwords_combined]
)

Total stopwords combinadas: 485


In [ ]:
# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')

In [ ]:
DF_BookingCol['Lematizado_Combined'] = DF_BookingCol['Sin_Stopwords_Combined'].apply(
    lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
)

# Ver el resultado
print(DF_BookingCol[['Lematizado_Combined','Sin_Stopwords_Combined']])

                                     Lematizado_Combined  \
0      [Deje, pago, servicio, transporte, aeropuerto,...   
1      [llegar, hacer, leer, uno, condición, hospedaj...   
2      [bar, disponible, encontrar, cabello, cama, de...   
3      [luz, servia, ascensor, casi, diar, estabamo, ...   
4      [Podria, mejorar, recomendación, huesped, pers...   
...                                                  ...   
14995  [encanto, quedar, solo, paso, lugar, ir, quede...   
14996  [cama, comodo, poder, descansar, comodamente, ...   
14997  [Hotel, pequeno, ubicado, cerca, centro, comer...   
14998  [buena, habitación, pequena, limpio, comodo, i...   
14999  [encanto, atencion, persona, amable, gentil, s...   

                                  Sin_Stopwords_Combined  
0      [Deje, pago, servicio, transporte, aeropuerto,...  
1      [llega, hacen, leer, unas, condiciones, hosped...  
2      [bar, disponible, encontramos, cabellos, cama,...  
3      [luz, servia, ascensor, casi, dia, e

## 4.4. Stemming y Lemantización

 __Para este caso de estudio se determinó que la manera más adecuada para reducir las palabras a su forma base es la lemmatización ya que con este método se considera su significado y contexto gramatical, lo cual es una ventaja para el análisis de los comentarios realizados por las personas en Booking__

In [ ]:
# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')

In [ ]:
DF_BookingCol['Lematizado_Combined'] = DF_BookingCol['Sin_Stopwords_Combined'].apply(
    lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
)

# Ver el resultado
print(DF_BookingCol[['Sin_Stopwords_Combined','Lematizado_Combined']])

                                  Sin_Stopwords_Combined  \
0      [Deje, pago, servicio, transporte, aeropuerto,...   
1      [llega, hacen, leer, unas, condiciones, hosped...   
2      [bar, disponible, encontramos, cabellos, cama,...   
3      [luz, servia, ascensor, casi, dia, estabamos, ...   
4      [Podria, mejorar, recomendaciones, huespedes, ...   
...                                                  ...   
14995  [encanto, quedaba, solo, pasos, lugar, iba, Qu...   
14996  [cama, comoda, pude, descansar, comodamente, c...   
14997  [Hotel, pequeno, ubicado, cerca, centro, comer...   
14998  [Buenas, habitaciones, pequenas, limpio, comod...   
14999  [encanto, atencion, personas, amables, gentile...   

                                     Lematizado_Combined  
0      [Deje, pago, servicio, transporte, aeropuerto,...  
1      [llegar, hacer, leer, uno, condición, hospedaj...  
2      [bar, disponible, encontrar, cabello, cama, de...  
3      [luz, servia, ascensor, casi, diar, 

# __5. Se aplica modelo de Bag of Words__

## 5.1. CountVectorizer

In [ ]:
# Convertir las listas de lemas a textos unificados
DF_BookingCol['Lematizado_unido'] = DF_BookingCol['Lematizado_Combined'].apply(lambda tokens: " ".join(tokens))

# Inicializar el vectorizador
vectorizer = CountVectorizer()

# Aplicar BoW a la columna 'Texto_Lematizado'
bow_matrix = vectorizer.fit_transform(DF_BookingCol['Lematizado_unido'])

# Convertir la matriz de BoW a un DataFrame para una visualización más fácil
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar el resultado
print(bow_df)

       aa  aaael  aan  aao  abajo  abalorio  abandonado  abandonar  \
0       0      0    0    0      0         0           0          0   
1       0      0    0    0      0         0           0          0   
2       0      0    0    0      0         0           0          0   
3       0      0    0    0      0         0           0          0   
4       0      0    0    0      0         0           0          0   
...    ..    ...  ...  ...    ...       ...         ...        ...   
14995   0      0    0    0      0         0           0          0   
14996   0      0    0    0      0         0           0          0   
14997   0      0    0    0      0         0           0          0   
14998   0      0    0    0      0         0           0          0   
14999   0      0    0    0      0         0           0          0   

       abandonarar  abanico  ...  zuleima  zully  zullys  zum  zón  án  él  \
0                0        0  ...        0      0       0    0    0   0   0   
1  

## 5.2. TF-IDF (Term Frequency-Inverse Document Frequency):

In [ ]:
# Unir los lemas en textos completos para cada fila
DF_BookingCol['Texto_Lematizado'] = DF_BookingCol['Lematizado_Combined'].apply(lambda tokens: " ".join(tokens))

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar los textos
tfidf_matrix = tfidf_vectorizer.fit_transform(DF_BookingCol['Texto_Lematizado'])

# Convertir la matriz a un DataFrame para visualizarla
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Mostrar los primeros resultados
print(tfidf_df.head())

    aa  aaael  aan  aao  abajo  abalorio  abandonado  abandonar  abandonarar  \
0  0.0    0.0  0.0  0.0    0.0       0.0         0.0        0.0          0.0   
1  0.0    0.0  0.0  0.0    0.0       0.0         0.0        0.0          0.0   
2  0.0    0.0  0.0  0.0    0.0       0.0         0.0        0.0          0.0   
3  0.0    0.0  0.0  0.0    0.0       0.0         0.0        0.0          0.0   
4  0.0    0.0  0.0  0.0    0.0       0.0         0.0        0.0          0.0   

   abanico  ...  zuleima  zully  zullys  zum  zón   án        él   ín   ón  \
0      0.0  ...      0.0    0.0     0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
1      0.0  ...      0.0    0.0     0.0  0.0  0.0  0.0  0.225922  0.0  0.0   
2      0.0  ...      0.0    0.0     0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
3      0.0  ...      0.0    0.0     0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
4      0.0  ...      0.0    0.0     0.0  0.0  0.0  0.0  0.000000  0.0  0.0   

    ús  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.

## 5.3. Identificación de frecuencia de las palabras:

In [ ]:
# Sumar las ocurrencias de cada palabra
palabra_frecuencias = bow_df.sum().sort_values(ascending=False)
print(palabra_frecuencias.head(9))  # Top 10 palabras más frecuentes

habitacion    5022
hotel         3899
personal      3208
desayuno      3153
excelente     2629
atencion      2580
ubicacion     2489
buen          2313
servicio      2138
dtype: int64


# __6. Conclusiones__